In [159]:
from enum import Enum
from itertools import product
from typing import List
from typing import Tuple
from typing import Union
import math

import matplotlib.pyplot as plt

In [4]:
# Constants
ALPHABET = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

SQUARE_3 = [
    [4, 9, 2],
    [3, 5, 7],
    [8, 1, 6],
]
SQUARE_4 = [
    [4, 14, 15, 1],
    [9, 7, 6, 12],
    [5, 11, 10, 8],
    [16, 2, 3, 13],
]
SQUARE_7 = [
    [22, 47, 16, 41, 10, 35, 4],
    [5, 23, 48, 17, 42, 11, 29],
    [30, 6, 24, 49, 18, 36, 12],
    [13, 31, 7, 25, 43, 19, 37],
    [38, 14, 32, 1, 26, 44, 20],
    [21, 39, 8, 33, 2, 27, 45],
    [46, 15, 40, 9, 34, 3, 28],
]

In [154]:
# Utilities
class Option(Enum):
    NEVER = 0
    MAYBE = 1
    ALWAYS = 2

class NumSeq:
    def __init__(self, numbers: Union[str, Tuple[int]]):
        """Construct from tuple of numbers or string"""
        if isinstance(numbers, str):
            self.numbers = self.get_numbers(numbers)
        else:
            self.numbers = tuple(numbers)

    @staticmethod
    def get_numbers(text: str) -> Tuple[int]:
        numbers = []
        for ch in text.upper():
            i = ALPHABET.find(ch)
            if i < 0:
                continue
    
            mag =  10 ** (i // 9)
            offset = (i % 9) + 1
            numbers.append(mag * offset)
        return tuple(numbers)

    def compatible(self, size: int) -> bool:
        """Check if sequence is compatible with square of given size"""
        for ch in self.numbers:
            if ch > size * size:
                return False

        return True

    # Create children

    def remove_duplicates(self, option: Option = Option.MAYBE) -> List:
        """Possibly remove duplicates"""
        if option == Option.NEVER:
            return [self]

        dupe_i = []
        for i, ch in enumerate(self.numbers):
            if ch in self.numbers[:i]:
                dupe_i.append(i)

        choices = [[True] * len(dupe_i)]
        if option == Option.MAYBE:
            choices = product([False, True], repeat=len(dupe_i))

        output = []
        for choice in choices:
            numbers = []
            last_i = 0
            for i, drop in zip(dupe_i, choice):
                numbers += self.numbers[last_i:i]
                if not drop:
                    numbers.append(self.numbers[i])
                last_i = i + 1
            numbers += self.numbers[last_i:]
            output.append(NumSeq(numbers))

        return output

    def _reduce_one(self, ch: int) -> List[int]:
        """Reduce one character"""
        output = {ch // 100, ch // 10, ch}
        output.discard(0)
        return list(sorted(output))

    def reduce(self, size: int, option: Option = Option.MAYBE) -> List:
        """Possibly reduce numbers by an order of magnitude"""
        if option == Option.NEVER:
            return [self]

        choices = [self._reduce_one(ch) for ch in self.numbers]

        if option == Option.ALWAYS:
            choices = [[choice[0]] for choice in choices]

        output = [NumSeq(choice) for choice in product(*choices)]
        output = [o for o in output if o.compatible(size)]

        return output

    def _combine_from(self, size: int, from_i: int) -> List:
        """Possibly combine adjacent numbers from given index"""
        if from_i + 1 >= len(self.numbers):
            return [self]

        x1 = self.numbers[from_i]
        x2 = self.numbers[from_i + 1]

        mag1 = math.floor(math.log(x1, 10))
        mag2 = math.floor(math.log(x2, 10))

        if mag1 <= mag2:
            return [self]

        # TODO
        
    
    def combine(self, size: int, option: Option = Option.MAYBE) -> List:
        """Possibly combine adjacent numbers (20 + 1 -> 21)."""

        # TODO
        

    # Rendering

    def draw(self, square: List[List[int]], show=True):
        """Draw a single sequence on a square"""
        coord_map = dict()
        size = len(square)
        for i, row in enumerate(square):
            for j, ch in enumerate(row):
                coord_map[ch] = (i + 0.5, j + 0.5)

        xs = []
        ys = []
        if self.compatible(size):
            xs = [coord_map[ch][0] for ch in self.numbers]
            ys = [coord_map[ch][1] for ch in self.numbers]

        plt.plot(xs, ys, 'o-r')

        plt.xticks([])
        plt.yticks([])
        plt.xlim([0, size + 1])
        plt.ylim([0, size + 1])

        if show:
            plt.show()

    @staticmethod
    def draw_all(sequences: List, square: List[List[int]]):
        """Draw all sequences on a square"""
        cols = 5
        rows = ((len(sequences) - 1) // cols) + 1
        print(cols, rows)
        
        plt.figure(figsize=(15, 15 * rows / cols))

        for i, seq in enumerate(sequences):
            plt.subplot(rows, cols, i + 1)
            seq.draw(square, show=False)

        plt.show()

    # Operators

    def __repr__(self) -> str:
        """repr operator"""
        return f'NumSeq({self.numbers})'

    def __hash__(self) -> int:
        """hash operator"""
        return hash(self.numbers)

    def __eq__(self, other) -> bool:
        """"Equality operator"""
        if not isinstance(other, NumSeq):
            return False
        return self.numbers == other.numbers